### Module import
In this code section the required modules are imported:
* __*date*__ to create a simulated NPI project. `Project`class requires milestone dates in `Date` class (format).
* __*matplotlib*__ to plot the fuzzy functions.
* __*numpy*__ to work with values in fuzzy variables.
* __*simulation*__ is a custom module with the required classes to simulate the NPI Sourcing process. The code can be found in `simulation.py` or `simulation.ipynb`.
* __*skfuzzy*__ is a fuzzy logic framework for Python.

In [1]:
from datetime import date
import matplotlib.pyplot as plt
import numpy as np
import simulation
import skfuzzy as fuzzy

## Generate training data frame

### Generate sample suppliers
Based in fuzzy variables (delivery time, quotation time, price, punctuality) and their values (low, regular, high), it will be necessary to generate 81 suppliers with different profiles each one, aiming to obtain a initial data frame with unbiased data.
Supplier/profile quantity is calculated as follows in this formula: $`R_{max} = \prod_{i=1}^{n} m_{i} = \prod_{i=1}^{4} 3 = 3^{4} = 81`$

In [6]:
env = simulation.Environment()  # Initialize environment for the simulation.

current_supplier_names = ("Tuberías ABC, S.A. de C.V.", "Tuberías DEF, S.A.", "Tuberías GHI, S. de R.L. de C.V.")
current_supplier_profile = (("regular", "low", "high", "high"), ("regular", "high", "low", "regular"), ("high", "high", "low", "low"))

for delivery_time in ["low", "regular", "high"]:
    for quotation_time in ["low", "regular", "high"]:
        for price in ["low", "regular", "high"]:
            for punctuality in ["low", "regular", "high"]:
                supplier_number = len(env.suppliers) + 1
                supplier_id = f"1{str(supplier_number).zfill(7)}"    # Supplier ID must be 8 character length string.

                if (delivery_time, quotation_time, price, punctuality) in current_supplier_profile:
                    supplier_index = current_supplier_profile.index((delivery_time, quotation_time, price, punctuality))
                    supplier_name = current_supplier_names[supplier_index]
                else:
                    supplier_name = f"Supplier {supplier_number}"   # Generate a supplier name concatenating Supplier and supplier number.

                supplier = simulation.Supplier(supplier_id, supplier_name, delivery_time, quotation_time, price, punctuality)
                env.add_supplier(supplier)  # Add supplier to simulation environment.

## Generate current suppliers

### Create project and ECNs
Create a sample project named *Alaska*.
It is important to define the milestone dates for this projects. Required milestone dates are:
* **Design freeze (DF):** Restrict not important changes to the design of the project. Most of ECNs (Engineering Change Notification) are released near this date.
* **MCS:** I do not remember what does it stands for in this moment. In this stage a prototype containing the exact sourced material for the project to review to review their feasibility and adjust parameters in case it is needed. In this stage, part numbers do not need to be PPAP approved and built units cannot be sold.
* **Pilot:** In this stage a small but significant quantity of units of the project are produced. It aims to test the whole manufacturing process. All the material needs to be PPAP approved in this stage. Pilot units may be sold under Marketing approval.
* **Start of production (SOP):** Project implementation. Suppliers need to be ready at least six weeks before this date, however, *supplier readiness* may be longer depending on material lead time.

A quantity of 30 ECNs is created for this initial data frame. Each ECN contains a variable quantity of part numbers of different kinds. The quantity and kinds are randomly generated taking into account real statistics.

The simulated part numbers are copper tubes. From a real data set it was determined six kinds three complexity degrees (low, medium, regular). Both parameters affect prices and quantities.

In [ ]:
alaska = simulation.Project(
    name="Alaska",
    df_date=date(2024,5,10), 
    mcs_date=date(2024,7,15),
    pilot_date=date(2024,9,1), 
    sop_date=date(2025,1,27)
)   # Create project Alaska.

env.gen_ecns(alaska, 81) # Create 30 ECNs.

### Generating data frame
Generate the data frame is divided in two steps:
1. Quoting all the ECNs with all the suppliers. The simulation uses real statistical data to generate random dates and prices.
2. Implementing each ECN with random chosen suppliers. A uniform distribution is used to randomly chose the suppliers.

In [8]:
env.quote_all_ecns()
env.gen_initial_item_master_df()

env.item_master # Display all the dataframe, with no filters.

/Users/rubusarbaro/.pyProjects/supplier-selection_fuzzy-logic_thesis/simulation.py:496: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2024-08-14' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  self.item_master.loc[
/Users/rubusarbaro/.pyProjects/supplier-selection_fuzzy-logic_thesis/simulation.py:496: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2024-08-21' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  self.item_master.loc[
/Users/rubusarbaro/.pyProjects/supplier-selection_fuzzy-logic_thesis/simulation.py:496: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2024-09-28' has dtype incompatible with float64, please explicitly cast to a compatible dtype first

,Project,ECN,ECN release,Part number,Complexity,EAU,Delivery profile,Quotation profile,Price profile,Punctuality profile,...,PPAP approval,Contract date,Quotation time,OTD,Delivery time,FY Spend,Awarded,MCS ready,Pilot ready,SOP ready
0,Alaska,ECN0000001,2024-07-19,A0B000001,medium,683.0,low,low,low,low,...,NaN,NaN,19.0,NaN,NaN,2096.81,0.0,NaN,NaN,NaN
1,Alaska,ECN0000001,2024-07-19,A0B000001,medium,683.0,low,low,low,regular,...,NaN,NaN,23.0,NaN,NaN,18215.61,False,NaN,NaN,NaN
2,Alaska,ECN0000001,2024-07-19,A0B000001,medium,683.0,low,low,low,high,...,NaN,NaN,7.2,NaN,NaN,39122.24,False,NaN,NaN,NaN
3,Alaska,ECN0000001,2024-07-19,A0B000001,medium,683.0,low,low,regular,low,...,NaN,NaN,27.0,NaN,NaN,53369.62,False,NaN,NaN,NaN
4,Alaska,ECN0000001,2024-07-19,A0B000001,medium,683.0,low,low,regular,regular,...,NaN,NaN,53.0,NaN,NaN,28009.83,False,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8095,Alaska,ECN0000030,2024-07-28,A0E000027,high,493.0,high,high,regular,regular,...,NaN,NaN,20.0,NaN,NaN,58223.30,False,NaN,NaN,NaN
8096,Alaska,ECN0000030,2024-07-28,A0E000027,high,493.0,high,high,regular,high,...,NaN,NaN,15.0,NaN,NaN,63049.77,False,NaN,NaN,NaN
8097,Alaska,ECN0000030,2024-07-28,A0E000027,high,493.0,high,high,high,low,...,NaN,NaN,83.0,NaN,NaN,69291.15,False,NaN,NaN,NaN
8098,Alaska,ECN0000030,2024-07-28,A0E000027,high,493.0,high,high,high,regular,...,NaN,NaN,23.0,NaN,NaN,31872.45,False,NaN,NaN,NaN


In [9]:
env.item_master.to_excel("test.xlsx")

## Generate data frame for fuzzy model
In this section a new project and data frame containing 10 copper piping ECNs is generated to apply it with the 

## Define parameters for fuzzy variables

In [ ]:
var_delivery_time = None
var_quotation_time = None
var_price = None
var_punctuality = None

In [ ]:
"""
Supplier 1 (B): Delivery: Regular, Quotation: Low (Good), Punctuality: High, Price: High
Supplier 2 (MS): Delivery: Regular, Quotation: High, Punctuality: Low, Price: Regular
Supplier 3 (TH): Delivery: High (Bad), Quotation: High, Punctuality: Low, Price: Low
"""